In [18]:
#[Problem 1] Simple Forward propagation implementation of RNN
import numpy as np

class SimpleRNN:
    def __init__(self, n_features, n_nodes):
        self.n_features = n_features
        self.n_nodes = n_nodes
        self.Wx = np.random.randn(n_features, n_nodes) * 0.01
        self.Wh = np.random.randn(n_nodes, n_nodes) * 0.01
        self.b = np.zeros((n_nodes,))
    
    def forward(self, x, h0=None):
        batch_size, n_sequences, _ = x.shape
        if h0 is None:
            h_t = np.zeros((batch_size, self.n_nodes))
        else:
            h_t = h0

        self.hs = []   
        self.as_ = []  
        for t in range(n_sequences):
            x_t = x[:, t, :]  # (batch_size, n_features)
            a_t = x_t @ self.Wx + h_t @ self.Wh + self.b
            h_t = np.tanh(a_t)
            
            self.hs.append(h_t)
            self.as_.append(a_t)

        return h_t, np.array(self.hs)


    #[Problem 3] (Advance assignment) Implementation of backpropagation

    def backward(self, x, dhs, lr=0.01):
        """
        Backpropagation Through Time (BPTT).
        x: (batch_size, n_sequences, n_features)
        dhs: lista de gradientes dL/dh_t vindos do output (mesma shape que self.hs)
        """
        batch_size, n_sequences, _ = x.shape
    
        dWx = np.zeros_like(self.Wx)
        dWh = np.zeros_like(self.Wh)
        db = np.zeros_like(self.b)
    
        dh_next = np.zeros((batch_size, self.n_nodes))
    
        for t in reversed(range(n_sequences)):
            dh = dhs[t] + dh_next  # soma do erro de saída + erro propagado
            da = dh * (1 - np.tanh(self.as_[t]) ** 2)  # (batch_size, n_nodes)
    
            x_t = x[:, t, :]
            h_prev = np.zeros((batch_size, self.n_nodes)) if t == 0 else self.hs[t-1]
    
            dWx += x_t.T @ da
            dWh += h_prev.T @ da
            db  += np.sum(da, axis=0)
    
            # Propaga para o tempo anterior
            dh_next = da @ self.Wh.T
    
        # Atualização
        self.Wx -= lr * dWx
        self.Wh -= lr * dWh
        self.b  -= lr * db
    
        return dWx, dWh, db






In [19]:
#[Problem 2] Experiment of forward propagation with small sequence

x = np.array([[[1, 2], [2, 3], [3, 4]]]) / 100
w_x = np.array([[1, 3, 5, 7],
                [3, 5, 7, 8]]) / 100
w_h = np.array([[1, 3, 5, 7],
                [2, 4, 6, 8],
                [3, 5, 7, 8],
                [4, 6, 8, 10]]) / 100
b = np.array([1, 1, 1, 1], dtype=np.float64)  # <<< Corrigido para float

batch_size, n_sequences, n_features = x.shape
n_nodes = w_x.shape[1]

# Criar RNN e forçar os pesos
rnn = SimpleRNN(n_features, n_nodes)
rnn.Wx = w_x.copy()
rnn.Wh = w_h.copy()
rnn.b  = b.copy()

# Forward
h_final, hs = rnn.forward(x)
print("🔹 Forward result (last h):")
print(h_final)
print("\nExpected:")
print(np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]))

# Backward com gradiente fake (todos 1s, simulando perda)
dhs = [np.ones_like(h) for h in hs]
dWx, dWh, db = rnn.backward(x, dhs, lr=0.1)

print("\n🔹 Gradientes calculados:")
print("dWx:\n", dWx)
print("dWh:\n", dWh)
print("db:\n", db)

print("\n🔹 Pesos atualizados:")
print("Wx:\n", rnn.Wx)
print("Wh:\n", rnn.Wh)
print("b:\n", rnn.b)

🔹 Forward result (last h):
[[0.79494228 0.81839002 0.83939649 0.85584174]]

Expected:
[[0.79494228 0.81839002 0.83939649 0.85584174]]

🔹 Gradientes calculados:
dWx:
 [[0.02325421 0.02151871 0.01988135 0.01863078]
 [0.03524769 0.03286036 0.03059886 0.0288961 ]]
dWh:
 [[0.58867635 0.53378663 0.48206685 0.4416354 ]
 [0.5968625  0.54113328 0.48864029 0.44759017]
 [0.60427244 0.54778349 0.49459073 0.45298073]
 [0.61012671 0.55303731 0.49929153 0.45723902]]
db:
 [1.19934775 1.13416539 1.07175132 1.02653129]

🔹 Pesos atualizados:
Wx:
 [[0.00767458 0.02784813 0.04801187 0.06813692]
 [0.02647523 0.04671396 0.06694011 0.07711039]]
Wh:
 [[-0.04886764 -0.02337866  0.00179332  0.02583646]
 [-0.03968625 -0.01411333  0.01113597  0.03524098]
 [-0.03042724 -0.00477835  0.02054093  0.03470193]
 [-0.02101267  0.00469627  0.03007085  0.0542761 ]]
b:
 [0.88006523 0.88658346 0.89282487 0.89734687]
